In [1]:
# Import all the libraries required

import soccerdata as sd
import pandas as pd
from tqdm import tqdm
import ScraperFC as sfc
import traceback

import warnings
import numpy as np
import socceraction.spadl as spadl

# from pyspark.sql import SparkSession
# from pyspark.sql.types import StructType, StructField, StringType
# import pyspark.pandas as ps

[04/09/24 16:27:28] INFO     No custom team name replacements found. You can configure these in       ]8;id=526314;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=841949;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#87\87]8;;\
                             C:\Users\ksbha\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=821739;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=93195;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_config.py#177\177]8;;\
                             C:\Users\ksbha\soccerdata\config\league_dict.json.                                    

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\pyspark\pandas\__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [2]:
import datetime

In [3]:
## Create a connection to Snowflake
from sqlalchemy import create_engine
import snowflake.connector

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.2), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(


[04/09/24 16:27:54] WARNING  Failed to import ArrowResult. No Apache Arrow result set format can be    ]8;id=765501;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=495762;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#94\94]8;;\
                             used. ImportError: DLL load failed while importing arrow_iterator: The                
                             specified procedure could not be found.                                               

In [4]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas

In [5]:
pd.set_option('display.max_columns', None)

In [6]:
# spark = SparkSession.builder \
#     .master("local") \
#     .appName("Gegenstats") \
#     .getOrCreate()

In [7]:
scraper = sfc.Understat()
def scrape_league_data(years):
    output_df = pd.DataFrame(columns=['№', 'Team', 'M', 'W', 'D', 'L', 'G', 'GA', 'PTS', 'xG', 'NPxG', 'xGA',
       'NPxGA', 'NPxGD', 'PPDA', 'OPPDA', 'DC', 'ODC', 'xPTS', 'season','league'])
    for league_year in league_years:
        try:
            ooper = scraper.scrape_league_table(year=league_year[1], league=league_year[0])
            season_val = league_year[1]
            ooper['season'] = int(str(int(str(season_val)[2:])-1)+str(season_val)[2:])
            if league_year[0] == 'EPL':
                ooper['league'] = 'ENG-Premier League'
            output_df = pd.concat([output_df, ooper])
            
        except Exception as e:
            print(f"An error occurred for the year {league_years}: {e}")
            traceback.print_exc()
    scraper.close()
    return output_df

league_years = [('EPL', 2024)]  # Replace with your list of years
out = scrape_league_data(league_years)

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:361: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  columns = pd.read_html(table)[0].columns
d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\ScraperFC\Understat.py:366: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(table)[0]
C:\Users\ksbha\AppData\Local\Temp\ipykernel_16412\1491577790.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, ooper])


In [8]:
understat_table = out[['league','Team','season', 'W', 'D', 'L','G','GA', 'xG', 'xGA','NPxG','NPxGA','PPDA',
                       'OPPDA','PTS','xPTS']]

In [9]:
understat_table.loc[understat_table['Team'] == 'Manchester United', 'Team'] = 'Manchester Utd'
understat_table.loc[understat_table['Team'] == 'Newcastle United', 'Team'] = 'Newcastle Utd'
understat_table.loc[understat_table['Team'] == 'Tottenham Hotspur', 'Team'] = 'Tottenham'
understat_table.loc[understat_table['Team'] == 'Wolverhampton Wanderers', 'Team'] = 'Wolves'
understat_table.loc[understat_table['Team'] == 'Nottingham Forest', 'Team'] = "Nott'ham Forest"
understat_table.loc[understat_table['Team'] == 'Leicester', 'Team'] = 'Leicester City'
understat_table.loc[understat_table['Team'] == 'Leeds', 'Team'] = 'Leeds United'
understat_table.loc[understat_table['Team'] == 'Sheffield United', 'Team'] = "Sheffield Utd"
understat_table.loc[understat_table['Team'] == 'Luton', 'Team'] = 'Luton Town'
understat_table.loc[understat_table['Team'] == 'Norwich', 'Team'] = 'Norwich City'

In [10]:
understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)
understat_table.reset_index(drop=True, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_16412\614021855.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.sort_values(by=['league','season','Team'], ascending=True, inplace=True)


In [11]:
understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',
                                'L': 'TEAM_LOSSES', 'xPTS':'TEAM_XPTS', 'PTS':'TEAM_PTS', 'season':'SEASON',
                                'NPxGA':'NPXG_AGAINST', 'xGA':'XG_AGAINST'}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_16412\2521636844.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  understat_table.rename(columns={'Team':'TEAM_NAME', 'W':'TEAM_WINS', 'D':'TEAM_DRAWS', 'league':'COMPETITION',


In [12]:
understat_table.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

Get the match data from Snowflake so that we only scrape for matches we don't have yet 

In [13]:
fbref = sd.FBref(leagues="ENG-Premier League", seasons=["2324"], no_cache=True)
fbref_team_standard = fbref.read_team_season_stats(stat_type='standard')
# fbref_team_standard_oppo = fbref.read_team_season_stats(stat_type='standard', opponent_stats=True)
fbref_team_shooting = fbref.read_team_season_stats(stat_type='shooting')
fbref_team_shooting_oppo = fbref.read_team_season_stats(stat_type='shooting', opponent_stats=True)
fbref_team_passing = fbref.read_team_season_stats(stat_type='passing')
fbref_team_passing_oppo = fbref.read_team_season_stats(stat_type='passing', opponent_stats=True)
fbref_team_defense = fbref.read_team_season_stats(stat_type='defense')
fbref_team_possession = fbref.read_team_season_stats(stat_type='possession')
fbref_team_misc = fbref.read_team_season_stats(stat_type='misc')
fbref_keeper = fbref.read_team_season_stats(stat_type='keeper')
fbref_schedule = fbref.read_schedule()
# fbref_lineups = fbref.read_lineup()

[04/09/24 16:28:58] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\FBref               ]8;id=911802;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=494575;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#92\92]8;;\

In [14]:
# Convert the date column to datetime format
fbref_schedule['date'] = pd.to_datetime(fbref_schedule['date'])
# fbref_schedule = fbref_schedule[fbref_schedule['date'] < datetime.datetime(2023, 12, 20)]

fbref_schedule = fbref_schedule[fbref_schedule['date'] < datetime.datetime.now().strftime('%Y-%m-%d')]

In [15]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [16]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
)

[04/09/24 16:41:27] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=992275;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=774976;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=735696;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=74020;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

                    INFO     Setting use_openssl_only mode to False                              ]8;id=379278;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=923026;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1121\1121]8;;\

In [17]:
cursor = conn.cursor()

In [18]:
def fetch_data(cursor, query):
    cursor.execute(query)
    rows = cursor.fetchall()
    column_names = [desc[0] for desc in cursor.description]
    return pd.DataFrame(rows, columns=column_names)

In [19]:
df_matches = fetch_data(cursor, 'SELECT * FROM MATCHES')

[04/09/24 16:41:33] INFO     query: [SELECT * FROM MATCHES]                                           ]8;id=752099;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=663686;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[04/09/24 16:41:34] INFO     query execution done                                                     ]8;id=707541;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=422344;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 595                                    ]8;id=82977;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=478918;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [20]:
df_matches = df_matches[df_matches['SEASON'] == 2324]

In [21]:
matches_to_scarpe = set(fbref_schedule['game_id']) - set(df_matches['MATCH_ID'])
matches_to_scarpe = list(matches_to_scarpe)

In [22]:
# ## Create empty dataframe with 
# pd.DataFrame(columns=['jersey_number', 'player', 'team', 'is_starter', 'position', 'minutes_played'], index=['league', 'season',
#                                                                                                               'game'])

In [23]:
for i, val in enumerate(matches_to_scarpe):
    if i == 0:
        fbref_lineups = fbref.read_lineup(match_id=val)
    else:
        fbref_lineups = pd.concat([fbref_lineups, fbref.read_lineup(match_id=val)])

[04/09/24 16:41:49] INFO     [1/1] Retrieving game with id=c6079c2f                                    ]8;id=772234;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=142533;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:42:05] INFO     [1/1] Retrieving game with id=e8f307a3                                    ]8;id=26980;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=400243;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:42:21] INFO     [1/1] Retrieving game with id=1f5973e2                                    ]8;id=489674;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=693813;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:42:37] INFO     [1/1] Retrieving game with id=2047cc32                                    ]8;id=755279;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=136535;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:42:53] INFO     [1/1] Retrieving game with id=d3850c26                                    ]8;id=516938;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=559594;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

...

[04/09/24 16:44:31] INFO     [1/1] Retrieving game with id=fca82852                                    ]8;id=691499;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=573054;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:44:47] INFO     [1/1] Retrieving game with id=4f0b227b                                    ]8;id=667623;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=618952;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:45:03] INFO     [1/1] Retrieving game with id=77d7e2d6                                    ]8;id=804753;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=541484;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:45:20] INFO     [1/1] Retrieving game with id=a9d2819f                                    ]8;id=748243;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=930124;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

[04/09/24 16:45:36] INFO     [1/1] Retrieving game with id=d57ffe1a                                    ]8;id=833407;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=66106;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#884\884]8;;\

In [24]:
fbref_lineups.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
fbref_lineups.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_16412\3773466821.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  fbref_lineups.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)


In [25]:
fbref_lineups.loc[fbref_lineups['team'] == 'Brighton & Hove Albion', 'team'] = 'Brighton'
fbref_lineups.loc[fbref_lineups['team'] == 'Manchester United', 'team'] = 'Manchester Utd'
fbref_lineups.loc[fbref_lineups['team'] == 'Newcastle United', 'team'] = 'Newcastle Utd'
fbref_lineups.loc[fbref_lineups['team'] == 'Nottingham Forest', 'team'] = "Nott'ham Forest"
fbref_lineups.loc[fbref_lineups['team'] == 'Sheffield United', 'team'] = "Sheffield Utd"
fbref_lineups.loc[fbref_lineups['team'] == 'Tottenham Hotspur', 'team'] = 'Tottenham'
fbref_lineups.loc[fbref_lineups['team'] == 'West Ham United', 'team'] = 'West Ham'
fbref_lineups.loc[fbref_lineups['team'] == 'Wolverhampton Wanderers', 'team'] = 'Wolves'

In [27]:
fbref_team_standard = fbref_team_standard.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
# fbref_team_standard_oppo = fbref_team_standard_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting = fbref_team_shooting.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_shooting_oppo = fbref_team_shooting_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing = fbref_team_passing.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_passing_oppo = fbref_team_passing_oppo.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_defense = fbref_team_defense.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_possession = fbref_team_possession.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_team_misc = fbref_team_misc.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_keeper = fbref_keeper.rename_axis(index={'team': 'TEAM_NAME', 'season': 'SEASON', 'league': 'COMPETITION'})
fbref_schedule = fbref_schedule.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
fbref_lineups = fbref_lineups.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})
fbref_lineups.rename({'team': 'TEAM_NAME'}, axis=1, inplace=True)

In [28]:
## Get rid of incomplete games (Thank God Tom Lockyear is good <3)
fbref_schedule = fbref_schedule.dropna(subset=['home_xg'])

## Get rid of 'vs ' in the TEAM_NAME column
fbref_team_passing_oppo.reset_index(inplace=True)
fbref_team_shooting_oppo.reset_index(inplace=True)

In [29]:
fbref_team_passing_oppo['TEAM_NAME'] = fbref_team_passing_oppo['TEAM_NAME'].str.replace('vs ', '')
fbref_team_shooting_oppo['TEAM_NAME'] = fbref_team_shooting_oppo['TEAM_NAME'].str.replace('vs ', '')

fbref_team_passing_oppo.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)
fbref_team_shooting_oppo.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [30]:
def make_season_integer(df):
    indexes = df.index.names
    df = df.reset_index()
    df['SEASON'] = df['SEASON'].astype(int)
    df = df.set_index(indexes)
    return df

fbref_team_standard = make_season_integer(fbref_team_standard)
# fbref_team_standard_oppo = make_season_integer(fbref_team_standard_oppo)
fbref_team_shooting = make_season_integer(fbref_team_shooting)
fbref_team_shooting_oppo = make_season_integer(fbref_team_shooting_oppo)
fbref_team_passing = make_season_integer(fbref_team_passing)
fbref_team_passing_oppo = make_season_integer(fbref_team_passing_oppo)
fbref_team_defense = make_season_integer(fbref_team_defense)
fbref_team_possession = make_season_integer(fbref_team_possession)
fbref_team_misc = make_season_integer(fbref_team_misc)
fbref_schedule = make_season_integer(fbref_schedule)
fbref_keeper = make_season_integer(fbref_keeper)
fbref_lineups = make_season_integer(fbref_lineups)

In [31]:
team_matches_played = fbref_team_standard['Playing Time'].MP
team_goals_scored = understat_table.G
team_npxG_90 = fbref_team_standard['Per 90 Minutes'].npxG
team_goals_conceded = understat_table.GA
# team_xG_against_90 = fbref_team_standard_oppo['Per 90 Minutes'].xG
team_shots = fbref_team_shooting['Standard'].Sh
team_shotsOT = fbref_team_shooting['Standard'].SoT
team_pass_completed = fbref_team_passing["Total"].Cmp
team_pass_attempted = fbref_team_passing["Total"].Att
team_tackles = fbref_team_defense.Tackles.Tkl
team_tackles_won = fbref_team_defense.Tackles.TklW
team_takeons_attempted = fbref_team_possession['Take-Ons'].Att
team_takeons_completed = fbref_team_possession['Take-Ons'].Succ
team_crossesintoPA = fbref_team_passing['CrsPA']
team_fouls_against = fbref_team_misc['Performance']['Fld']
team_fouls_made = fbref_team_misc['Performance']['Fls']
team_interceptions = fbref_team_defense['Int']
team_blocks_shots = fbref_team_defense['Blocks'].Sh
team_blocks_pass = fbref_team_defense['Blocks'].Pass
team_clearances = fbref_team_defense.Clr
team_aerials_won = fbref_team_misc['Aerial Duels'].Won
team_aerials_lost = fbref_team_misc['Aerial Duels'].Lost
team_shots_faced = fbref_team_shooting_oppo['Standard'].Sh

In [32]:
team_recoveries= fbref_team_misc['Performance']['Recov']

In [33]:
teams_poss_won = team_interceptions + team_tackles_won + team_recoveries + team_blocks_shots + \
    team_blocks_pass + team_aerials_won
team_OPPDA = understat_table['OPPDA']
team_final_3rd_passes_against =  fbref_team_passing_oppo['1/3']
team_cleansheets = fbref_keeper['Performance']['CS']

In [34]:
team_names = fbref_team_standard.reset_index()[['TEAM_NAME','url']]
team_names['TEAM_FBREF_ID'] = team_names['url'].apply(lambda x: x.split('/')[3])      ## Extracting team_id from URL
team_names.drop(columns=['url'], inplace=True)
team_names.columns = team_names.columns.droplevel(1)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_16412\2026419047.py:3: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  team_names.drop(columns=['url'], inplace=True)


In [35]:
team_names['TEAM_FBREF_ID'] = team_names['TEAM_FBREF_ID'].astype('string')

In [36]:
team_names.drop_duplicates(inplace=True)

## WhoScored Data

In [37]:
ws = sd.WhoScored(leagues='ENG-Premier League', seasons=["2324"])

[04/09/24 16:47:26] INFO     Saving cached data to C:\Users\ksbha\soccerdata\data\WhoScored           ]8;id=557027;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=215977;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\_common.py#92\92]8;;\

[04/09/24 16:47:27] INFO     patching driver executable                                              ]8;id=539895;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\undetected_chromedriver\patcher.py\patcher.py]8;;\:]8;id=533002;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\undetected_chromedriver\patcher.py#346\346]8;;\
                             C:\Users\ksbha\appdata\roaming\undetected_chromedriver\undetected_chrom               
                             edriver.exe                                                                           

In [39]:
ws_schedule = ws.read_schedule()

[04/09/24 16:50:31] INFO     Scraping game schedule from                                           ]8;id=156357;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=647964;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#428\428]8;;\
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

[04/09/24 16:50:33] INFO     Scraping game schedule for Tuesday, Apr 2 2024                        ]8;id=21143;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=823451;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

[04/09/24 16:50:36] INFO     Scraping game schedule for Wednesday, Apr 3 2024                      ]8;id=534535;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=526645;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

[04/09/24 16:50:37] INFO     Scraping game schedule for Thursday, Apr 4 2024                       ]8;id=774914;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=748425;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

                    INFO     Scraping game schedule for Saturday, Apr 6 2024                       ]8;id=760446;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=419380;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

...

[04/09/24 16:52:03] INFO     Scraping game schedule for Sunday, Aug 20 2023                        ]8;id=534615;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=385176;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

                    INFO     Scraping game schedule for Monday, Aug 21 2023                        ]8;id=562259;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=331921;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

                    INFO     Scraping game schedule for Friday, Aug 25 2023                        ]8;id=588106;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=372581;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

                    INFO     Scraping game schedule for Saturday, Aug 26 2023                      ]8;id=283604;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=135941;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

[04/09/24 16:52:04] INFO     Scraping game schedule for Sunday, Aug 27 2023                        ]8;id=913053;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=782093;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#328\328]8;;\

In [40]:
ws_schedule = ws_schedule[ws_schedule['date'] < datetime.datetime.now().strftime('%Y-%m-%d')]

In [41]:
fbref_schedule = fbref_schedule.dropna(subset=['home_xg'])

In [42]:
ws_schedule = ws_schedule.reset_index().rename({'league':"COMPETITION", "season":"SEASON"}, axis=1)
ws_schedule.set_index(['COMPETITION', 'SEASON', 'game'], inplace=True)

In [43]:
ws_schedule.loc[ws_schedule['home_team'] == 'Manchester United', 'home_team'] = 'Manchester Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Manchester United', 'away_team'] = 'Manchester Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Newcastle United', 'home_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Newcastle United', 'away_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Newcastle', 'home_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['away_team'] == 'Newcastle', 'away_team'] = 'Newcastle Utd'
ws_schedule.loc[ws_schedule['home_team'] == 'Tottenham Hotspur', 'home_team'] = 'Tottenham'
ws_schedule.loc[ws_schedule['away_team'] == 'Tottenham Hotspur', 'away_team'] = 'Tottenham'
ws_schedule.loc[ws_schedule['home_team'] == 'Wolverhampton Wanderers', 'home_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['away_team'] == 'Wolverhampton Wanderers', 'away_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['home_team'] == 'Wolverhampton', 'home_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['away_team'] == 'Wolverhampton', 'away_team'] = 'Wolves'
ws_schedule.loc[ws_schedule['home_team'] == 'Nottingham Forest', 'home_team'] = "Nott'ham Forest"
ws_schedule.loc[ws_schedule['away_team'] == 'Nottingham Forest', 'away_team'] = "Nott'ham Forest"
ws_schedule.loc[ws_schedule['home_team'] == 'Leicester', 'home_team'] = "Leicester City"
ws_schedule.loc[ws_schedule['away_team'] == 'Leicester', 'away_team'] = "Leicester City"
ws_schedule.loc[ws_schedule['home_team'] == 'Leeds', 'home_team'] = "Leeds United"
ws_schedule.loc[ws_schedule['away_team'] == 'Leeds', 'away_team'] = "Leeds United"
ws_schedule.loc[ws_schedule['home_team'] == 'Sheffield United', 'home_team'] = "Sheffield Utd"
ws_schedule.loc[ws_schedule['away_team'] == 'Sheffield United', 'away_team'] = "Sheffield Utd"
ws_schedule.loc[ws_schedule['home_team'] == 'Luton', 'home_team'] = "Luton Town"
ws_schedule.loc[ws_schedule['away_team'] == 'Luton', 'away_team'] = "Luton Town"
ws_schedule.loc[ws_schedule['home_team'] == 'Norwich', 'home_team'] = "Norwich City"
ws_schedule.loc[ws_schedule['away_team'] == 'Norwich', 'away_team'] = "Norwich City"

In [44]:
ws_schedule.reset_index(inplace=True)
fbref_schedule.reset_index(inplace=True)

In [45]:
ws_schedule = ws_schedule.rename({'game_id':"ws_game_id"}, axis=1)

In [46]:
ws_schedule['SEASON'] = ws_schedule['SEASON'].astype('int64')

In [47]:
fbref_schedule = pd.merge(fbref_schedule, ws_schedule[["COMPETITION", "SEASON", "home_team", "away_team","ws_game_id"]], 
          on=["COMPETITION", "SEASON", "home_team", "away_team"], how='inner')

In [47]:
loader = ws.read_events(output_fmt='loader')

[02/20/24 11:14:51] INFO     Scraping game schedule from                                           ]8;id=386696;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=27624;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#427\427]8;;\
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

                    INFO     Scraping game schedule for Thursday, Feb 1 2024                       ]8;id=207140;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=9736;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Feb 3 2024                       ]8;id=68841;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=965766;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Sunday, Feb 4 2024                         ]8;id=866007;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=154424;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/20/24 11:14:52] INFO     Scraping game schedule for Monday, Feb 5 2024                         ]8;id=355530;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=211887;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

...

                    INFO     [255/259] Retrieving game with id=1729212                             ]8;id=972318;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=849531;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [256/259] Retrieving game with id=1729316                             ]8;id=121809;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=914749;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

[02/20/24 11:18:37] INFO     [257/259] Retrieving game with id=1729525                             ]8;id=463009;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=583198;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [258/259] Retrieving game with id=1729517                             ]8;id=853288;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=141776;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

                    INFO     [259/259] Retrieving game with id=1729526                             ]8;id=902866;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=622774;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

In [48]:
game_ids = fbref_schedule[fbref_schedule['game_id'].isin(matches_to_scarpe)]['ws_game_id'].unique()

In [50]:
len(game_ids)

15

In [50]:
ws_players = pd.DataFrame(columns=['game_id', 'team_id', 'player_id', 'player_name', 
                                   'is_starter','minutes_played', 'jersey_number', 'starting_position'])

In [51]:
for game in tqdm(game_ids):
    ws_players = pd.concat([ws_players, loader.players(game_id=game)])

100%|██████████| 10/10 [00:02<00:00,  4.91it/s]


In [52]:
ws_players.head()

,game_id,team_id,player_id,player_name,is_starter,minutes_played,jersey_number,starting_position
0,1729481,189,115131,Mark Flekken,True,106,1,GK
1,1729481,189,372724,Nathan Collins,True,106,22,DC
2,1729481,189,321085,Kristoffer Ajer,True,106,20,DC
3,1729481,189,94935,Ben Mee,True,106,16,DC
4,1729481,189,329554,Mads Roerslev,True,71,30,DMR


In [53]:
## WhoScored decide to name Willy Boly with two different names. Idiots
ws_players.replace({"Willy-Arnaud Boly": "Willy Boly"}, inplace=True)
ws_players.replace({"Kepa": "Kepa Arrizabalaga"}, inplace=True)
ws_players.replace({"Yehor Yarmolyuk": "Yehor Yarmoliuk"}, inplace=True)
ws_players.replace({"Vitalii Mykolenko": "Vitaliy Mykolenko"}, inplace=True)
ws_players.replace({"Dan Bentley": "Daniel Bentley"}, inplace=True)
ws_players.replace({"Toti": "Toti Gomes"}, inplace=True)
ws_players.replace({"Joe Hodge": "Joseph Hodge"}, inplace=True)
ws_players.replace({"Jonny Otto": "Jonny"}, inplace=True)
ws_players.replace({"Álvaro Carreras": "Álvaro Fernández"}, inplace=True)
ws_players.replace({"Alisson Becker": "Alisson"}, inplace=True)
ws_players.replace({"Andy Robertson": "Andrew Robertson"}, inplace=True)
ws_players.replace({"Konstantinos Tsimikas": "Kostas Tsimikas"}, inplace=True)
ws_players.replace({"Alphonse Aréola": "Alphonse Areola"}, inplace=True)
ws_players.replace({"Pape Sarr": "Pape Matar Sarr"}, inplace=True)
ws_players.replace({"Yago Alonso": "Yago Santiago"}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\3445675085.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ws_players.replace({"Willy-Arnaud Boly": "Willy Boly"}, inplace=True)


## Snowflake Data Upload

In [54]:
SNOWFLAKE_USER = 'kbharaj3'
SNOWFLAKE_PASSWORD = 'Snowfl@key0014'
SNOWFLAKE_ACCOUNT = 'qx25653.ca-central-1.aws'
SNOWFLAKE_WAREHOUSE = 'FOOTY_STORE'
SNOWFLAKE_DATABASE = 'GEGENSTATS'
SNOWFLAKE_SCHEMA = 'TABLES'

In [55]:
conn = snowflake.connector.connect(
    user=SNOWFLAKE_USER,
    password=SNOWFLAKE_PASSWORD,
    account=SNOWFLAKE_ACCOUNT,
    warehouse=SNOWFLAKE_WAREHOUSE,
    database=SNOWFLAKE_DATABASE,
    schema=SNOWFLAKE_SCHEMA
    )

[02/20/24 11:19:11] INFO     Snowflake Connector for Python Version: 3.3.1, Python Version:       ]8;id=193242;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=424805;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#334\334]8;;\
                             3.11.5, Platform: Windows-10-10.0.22000-SP0                                           

                    INFO     This connection is in OCSP Fail Open Mode. TLS Certificates would   ]8;id=672161;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py\connection.py]8;;\:]8;id=419145;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\connection.py#1103\1103]8;;\
                             be checked for validity and revocation status. Any other                              
                             Certificate Revocation related exceptions or OCSP Responder                           
                             failures would be disregarded in favor of connectivity.                               

In [56]:
cur = conn.cursor()
cur.execute(f"USE WAREHOUSE {SNOWFLAKE_WAREHOUSE}")

[02/20/24 11:19:12] INFO     query: [USE WAREHOUSE FOOTY_STORE]                                       ]8;id=455677;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=746174;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=725983;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=396551;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=165675;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=400583;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [57]:
create_schema_sql = f"CREATE SCHEMA IF NOT EXISTS {SNOWFLAKE_SCHEMA}"
cur.execute(create_schema_sql)

                    INFO     query: [CREATE SCHEMA IF NOT EXISTS TABLES]                              ]8;id=832567;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=958795;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=257605;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=832438;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=152204;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=738363;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [58]:
print("The Snowflake warehouse being used is:", SNOWFLAKE_WAREHOUSE)

The Snowflake warehouse being used is: FOOTY_STORE


In [59]:
cur.execute('SELECT * FROM TEAMS')
team_rows = cur.fetchall()
column_names = [desc[0] for desc in cur.description]
curr_team_names = pd.DataFrame(team_rows, columns=column_names)

[02/20/24 11:19:16] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=510843;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=25369;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

[02/20/24 11:19:17] INFO     query execution done                                                     ]8;id=979059;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=647396;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=707640;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=633875;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [60]:
## Below (if not blank) are the teams you need to create LOGOS for
team_names_not_in_curr = team_names[~team_names['TEAM_NAME'].isin(curr_team_names['TEAM_NAME'])]
team_names_not_in_curr

,TEAM_NAME,TEAM_FBREF_ID


In [61]:
def upsert_to_snowflake(table_name, dataframe, primary_keys):
    if isinstance(primary_keys, str):
        primary_keys = [primary_keys]

    temp_table_name = f"{table_name}_TEMP"
    success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)

    on_condition = ' AND '.join([f"{table_name}.{pk} = {temp_table_name}.{pk}" for pk in primary_keys])

    update_columns = [col for col in dataframe.columns if col not in primary_keys]
    update_sql = ', '.join([f"{table_name}.{col} = {temp_table_name}.{col}" for col in update_columns])

    merge_sql = f"""
    MERGE INTO {table_name} USING {temp_table_name}
    ON {on_condition}
    """

    if update_columns:
        merge_sql += f"""
        WHEN MATCHED THEN
            UPDATE SET
                {update_sql}
        """

    merge_sql += f"""
    WHEN NOT MATCHED THEN
        INSERT ({', '.join(dataframe.columns)})
        VALUES ({', '.join([f"{temp_table_name}.{col}" for col in dataframe.columns])})
    """

    cur.execute(merge_sql)
    cur.execute(f"DROP TABLE IF EXISTS {temp_table_name}")

    print(f"Upsert completed. {nrows} rows processed.")

In [62]:
# upsert_to_snowflake("TEAMS", team_names, ['TEAM_FBREF_ID'])

In [63]:
team_names = fetch_data(cur, 'SELECT * FROM TEAMS')

[02/20/24 11:19:23] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=20301;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=819806;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=334252;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=437328;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=136385;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=665565;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [64]:
team_names.drop(columns=['TEAM_LOGO_URL'], inplace=True)

In [65]:
team_standard_stats = understat_table.copy()

In [66]:
# team_standard_stats.reset_index(inplace=True)
# team_standard_stats.loc[team_standard_stats['TEAM_NAME'] == 'Luton', 'TEAM_NAME'] = 'Luton Town'
# team_standard_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [67]:
team_standard_stats['MATCHES_PLAYED'] = team_matches_played

In [68]:
team_standard_stats.reset_index(inplace=True)

In [69]:
team_standard_stats = team_standard_stats.merge(team_names, on='TEAM_NAME', how='left')

In [70]:
team_standard_stats = team_standard_stats[['TEAM_FBREF_ID','SEASON','COMPETITION','MATCHES_PLAYED','TEAM_WINS', 
                                           'TEAM_DRAWS', 'TEAM_LOSSES', 'TEAM_PTS', 'TEAM_XPTS']]

In [71]:
team_standard_stats

,TEAM_FBREF_ID,SEASON,COMPETITION,MATCHES_PLAYED,TEAM_WINS,TEAM_DRAWS,TEAM_LOSSES,TEAM_PTS,TEAM_XPTS
0,18bb7c10,2324,ENG-Premier League,25,17,4,4,55,52.58
1,8602292d,2324,ENG-Premier League,25,15,4,6,49,42.76
2,4ba7cbea,2324,ENG-Premier League,24,7,7,10,28,32.58
3,cd051869,2324,ENG-Premier League,24,7,4,13,25,35.49
4,d07537b9,2324,ENG-Premier League,25,10,8,7,38,38.6
5,943e8050,2324,ENG-Premier League,25,3,4,18,13,20.43
6,cff3d9bb,2324,ENG-Premier League,25,10,5,10,35,40.84
7,47c64c55,2324,ENG-Premier League,25,6,7,12,25,27.93
8,d3fd31cc,2324,ENG-Premier League,25,8,6,11,30,38.88
9,fd962109,2324,ENG-Premier League,25,8,5,12,29,28.03


In [72]:
upsert_to_snowflake("TEAM_STANDARD_STATS", team_standard_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/20/24 11:19:34] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=13850;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=451332;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=763599;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=369516;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=677373;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=327031;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

[02/20/24 11:19:35] INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=321740;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=107866;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=59246;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=252776;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:19:39] INFO     query execution done                                                     ]8;id=505488;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=665020;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_STANDARD_STATS_TEMP]                   ]8;id=898968;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=379329;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=440281;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=423444;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=873408;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=820446;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [73]:
team_attacking_stats = understat_table.copy()

In [74]:
team_attacking_stats = team_attacking_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [75]:
team_attacking_stats = team_attacking_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON','NPxG','xG']]

In [76]:
team_attacking_stats.rename(columns={'NPxG': 'NPXG', 'xG':'XG'}, inplace=True)

In [77]:
team_attacking_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [78]:
team_attacking_stats['GOALS_SCORED'] = team_goals_scored
team_attacking_stats['SHOTS'] = team_shots
team_attacking_stats['SHOTS_ON_TARGET'] = team_shotsOT
team_attacking_stats['PASS_COMPLETED'] = team_pass_completed
team_attacking_stats['PASS_ATTEMPTED'] = team_pass_attempted
team_attacking_stats['TAKEONS_ATTEMPTED'] = team_takeons_attempted
team_attacking_stats['TAKEONS_COMPLETED'] = team_takeons_completed
team_attacking_stats['CROSSES_INTO_PA'] = team_crossesintoPA
team_attacking_stats['FOULS_AGAINST'] = team_fouls_against

In [79]:
team_attacking_stats.reset_index(inplace=True)
team_attacking_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [81]:
upsert_to_snowflake("TEAM_ATTACKING_STATS", team_attacking_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/20/24 11:19:55] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=109969;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=858517;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=304727;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=514458;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=624177;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=528831;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=993373;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=61232;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/20/24 11:19:56] INFO     query execution done                                                     ]8;id=268926;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=123861;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:19:59] INFO     query execution done                                                     ]8;id=713218;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=334516;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_ATTACKING_STATS_TEMP]                  ]8;id=925382;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=860835;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=751111;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=948955;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=181857;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=316429;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [82]:
team_defending_stats = understat_table.copy()

In [83]:
team_defending_stats = team_defending_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [84]:
team_defending_stats = team_defending_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [85]:
team_defending_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [86]:
team_defending_stats['GOALS_CONCEDED'] = team_goals_conceded
team_defending_stats['XG_AGAINST'] = understat_table['XG_AGAINST']
team_defending_stats['NPXG_AGAINST'] = understat_table['NPXG_AGAINST']
team_defending_stats['TACKLES'] = team_tackles
team_defending_stats['TACKLES_WON'] = team_tackles_won
team_defending_stats['FOULS_MADE'] = team_fouls_made
team_defending_stats['INTERCEPTIONS'] = team_interceptions
team_defending_stats['BLOCKED_SHOTS'] = team_blocks_shots
team_defending_stats['BLOCKED_PASSES'] = team_blocks_pass
team_defending_stats['CLEARANCES'] = team_clearances
team_defending_stats['PPDA'] = understat_table['PPDA']
team_defending_stats['OPP_PPDA'] = team_OPPDA
team_defending_stats['POSS_WON'] = teams_poss_won
team_defending_stats['FINAL_3RD_PASSES_AGAINST'] = team_final_3rd_passes_against
team_defending_stats['CLEAN_SHEETS'] = team_cleansheets
team_defending_stats['SHOTS_FACED'] = team_shots_faced

In [87]:
team_defending_stats.reset_index(inplace=True)
team_defending_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [88]:
# sql_command = """
# ALTER TABLE GEGENSTATS.TABLES.TEAM_DEFENDING_STATS 
# ADD COLUMN SHOTS_FACED NUMBER(38,0);
# """

# cur.execute(sql_command)

In [89]:
# team_shots_faced = team_shots_faced.reset_index()

In [90]:
# for index, row in team_shots_faced.iterrows():
#     sql_command = """
#     UPDATE GEGENSTATS.TABLES.TEAM_DEFENDING_STATS
#     SET SHOTS_FACED = %s
#     WHERE TEAM_FBREF_ID = (SELECT TEAM_FBREF_ID FROM GEGENSTATS.TABLES.TEAMS WHERE TEAM_NAME = %s) 
#       AND COMPETITION = %s
#       AND SEASON = %s
#     """

#     values = (row['Sh'], row['TEAM_NAME'], row['COMPETITION'], row['SEASON'])

#     cur.execute(sql_command, values)

In [91]:
team_defending_stats.head()

,COMPETITION,SEASON,TEAM_FBREF_ID,GOALS_CONCEDED,XG_AGAINST,NPXG_AGAINST,TACKLES,TACKLES_WON,FOULS_MADE,INTERCEPTIONS,BLOCKED_SHOTS,BLOCKED_PASSES,CLEARANCES,PPDA,OPP_PPDA,POSS_WON,FINAL_3RD_PASSES_AGAINST,CLEAN_SHEETS,SHOTS_FACED
0,ENG-Premier League,2324,18bb7c10,22,21.7,20.18,411,230,239,188,75,211,325,9.06,17.36,2204,537,10,214
1,ENG-Premier League,2324,8602292d,33,37.57,37.57,360,213,267,173,65,187,386,12.59,10.85,1991,585,6,266
2,ENG-Premier League,2324,4ba7cbea,46,41.55,37.59,452,262,316,223,96,214,492,9.87,8.29,2446,689,5,341
3,ENG-Premier League,2324,cd051869,43,37.11,36.35,425,256,246,236,80,197,582,10.16,10.49,2374,978,4,336
4,ENG-Premier League,2324,d07537b9,40,39.48,35.68,407,240,269,198,94,190,391,9.14,17.82,2209,594,3,296


In [92]:
upsert_to_snowflake("TEAM_DEFENDING_STATS", team_defending_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/20/24 11:20:12] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=359257;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=385577;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=931604;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=201927;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=938482;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=551908;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=745615;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=319005;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=722926;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=386232;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:20:15] INFO     query execution done                                                     ]8;id=199940;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=349710;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_DEFENDING_STATS_TEMP]                  ]8;id=566599;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=399335;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=368998;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=458165;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=55344;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=27612;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [93]:
team_misc_stats = understat_table.copy()

In [94]:
team_misc_stats = team_misc_stats.reset_index().merge(team_names, on='TEAM_NAME', how='left')

In [95]:
team_misc_stats = team_misc_stats[['TEAM_NAME','TEAM_FBREF_ID','COMPETITION','SEASON']]

In [96]:
team_misc_stats.set_index(['COMPETITION', 'SEASON', 'TEAM_NAME'], inplace=True)

In [97]:
team_misc_stats['AERIALS_WON'] = team_aerials_won
team_misc_stats['AERIALS_LOST'] = team_aerials_lost

In [98]:
team_misc_stats.reset_index(inplace=True)
team_misc_stats.drop(columns=['TEAM_NAME'], inplace=True)

In [99]:
upsert_to_snowflake("TEAM_MISC_STATS", team_misc_stats, ['TEAM_FBREF_ID', 'COMPETITION','SEASON'])

[02/20/24 11:20:18] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=670667;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=664854;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=368632;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=291804;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=904196;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=695247;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=285752;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=146919;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=288800;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=36373;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

                    INFO     query execution done                                                     ]8;id=630126;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=206188;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS TEAM_MISC_STATS_TEMP]                       ]8;id=851823;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=939991;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=521564;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=663069;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=184047;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=408598;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


## Competition, Season and Match Data

In [100]:
df_competitions = fbref_schedule.reset_index()[['COMPETITION', 'SEASON']].drop_duplicates().copy()

In [101]:
df_competitions['COMPETITION_NAME'] = ''
df_competitions['COMPETITION_ACRONYM'] = ''

In [102]:
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_NAME'] = 'English Premier League'
df_competitions.loc[df_competitions['COMPETITION'] == 'ENG-Premier League', 'COMPETITION_ACRONYM'] = 'EPL'

In [103]:
df_competitions

,COMPETITION,SEASON,COMPETITION_NAME,COMPETITION_ACRONYM
0,ENG-Premier League,2324,English Premier League,EPL


In [726]:
upsert_to_snowflake("COMPETITIONS", df_competitions, ['COMPETITION','SEASON'])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\138046206.py:6: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)


[02/03/24 00:22:14] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=584206;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=6003;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=707377;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=627592;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=666461;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=652715;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=213671;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=749441;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

...

[02/03/24 00:22:17] INFO     query execution done                                                     ]8;id=578402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=339994;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS COMPETITIONS_TEMP]                          ]8;id=805650;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=948061;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=832907;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=692539;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=75225;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=643447;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 1 rows processed.


In [104]:
df_seasons = df_competitions[['SEASON']].drop_duplicates().copy()

In [105]:
df_seasons['SEASON_NAME'] = ''
df_seasons['START_YEAR'] = 0
df_seasons['END_YEAR'] = 0

In [106]:
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'SEASON_NAME'] = '2021-2022'
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'START_YEAR'] = 2021
# df_seasons.loc[df_seasons['SEASON'] == 2122, 'END_YEAR'] = 2022
df_seasons.loc[df_seasons['SEASON'] == 2324, 'SEASON_NAME'] = '2023-2024'
df_seasons.loc[df_seasons['SEASON'] == 2324, 'START_YEAR'] = 2023
df_seasons.loc[df_seasons['SEASON'] == 2324, 'END_YEAR'] = 2024

In [107]:
df_seasons

,SEASON,SEASON_NAME,START_YEAR,END_YEAR
0,2324,2023-2024,2023,2024


In [731]:
upsert_to_snowflake("SEASONS", df_seasons, ['SEASON'])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_6388\138046206.py:6: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  success, nchunks, nrows, _ = write_pandas(conn, dataframe, temp_table_name, auto_create_table=True)


[02/03/24 00:22:36] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=960012;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=24236;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=187255;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=848075;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=453456;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=544985;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=987055;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=480663;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

...

                    INFO     query execution done                                                     ]8;id=755598;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=990214;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS SEASONS_TEMP]                               ]8;id=149644;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=558371;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=608184;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=709220;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=183155;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=359641;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 1 rows processed.


In [108]:
df_stadiums = fbref_schedule.reset_index()[['home_team', 'SEASON',
            'venue']].drop_duplicates().rename(columns={'home_team': 'TEAM_NAME', 'venue': 'STADIUM'})

In [109]:
df_stadiums = df_stadiums.merge(team_names, on='TEAM_NAME')[['STADIUM','SEASON','TEAM_FBREF_ID']]

In [111]:
upsert_to_snowflake("STADIUMS", df_stadiums, ['TEAM_FBREF_ID','STADIUM', 'SEASON'])

[02/20/24 11:20:50] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=926378;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=312979;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=718140;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=614734;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=623370;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=380377;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=676465;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=212366;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=827321;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=237987;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:20:53] INFO     query execution done                                                     ]8;id=361452;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=450447;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS STADIUMS_TEMP]                              ]8;id=614536;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=469280;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=505841;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=864402;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=404539;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=248589;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 20 rows processed.


In [112]:
df_matches = fbref_schedule.reset_index()[['game_id', 'date', 'time', 'home_team', 'away_team', 'COMPETITION', 'SEASON', 'venue',
                                           'week', 'day', 'score', 'home_xg', 'away_xg', 'attendance', 'referee', 'ws_game_id']]

In [113]:
df_matches = df_matches.merge(team_names[['TEAM_FBREF_ID','TEAM_NAME']].rename(columns={'TEAM_FBREF_ID': 'home_team_id', 
                                                         'TEAM_NAME': 'home_team'}), on='home_team', how='left')

In [114]:
df_matches = df_matches.merge(team_names[['TEAM_FBREF_ID','TEAM_NAME']].rename(columns={'TEAM_FBREF_ID': 'away_team_id', 
                                                         'TEAM_NAME': 'away_team'}), on='away_team', how='left')

In [116]:
# Combine df_matches date and time to a dt.datatime configuration
df_matches['date'] = df_matches['date'].astype(str)
df_matches['time'] = df_matches['time'].astype(str)
df_matches['date_time'] = df_matches['date'] + ' ' + df_matches['time']
df_matches['date_time'] = pd.to_datetime(df_matches['date_time'])

In [117]:
# Split score in df_matches by "-" to home_team_score and away_team_score 
df_matches[['home_team_score', 'away_team_score']] = df_matches['score'].str.split('–', expand=True)

In [118]:
df_matches.rename(columns={'game_id': 'MATCH_ID', 'date_time': 'DATE_TIME', 'home_team_id': 'HOME_TEAM_ID', 'away_team_id': 'AWAY_TEAM_ID',
                           'venue': 'STADIUM', 'week': 'GAMEWEEK', 'day': 'DAY','home_team_score': 'HOME_TEAM_SCORE', 
                           'away_team_score':'AWAY_TEAM_SCORE', 'home_xg':'HOME_TEAM_XG','away_xg':'AWAY_TEAM_XG',
                           'attendance':'ATTENDANCE', 'referee': 'REFEREE', 'ws_game_id':'WS_MATCH_ID'}, inplace=True)

In [119]:
df_matches.drop(columns=['date', 'time', 'home_team', 'away_team', 'score'], inplace=True)

In [120]:
df_matches['DATE_TIME'] = df_matches['DATE_TIME'].astype('string')

In [122]:
upsert_to_snowflake("MATCHES", df_matches, ['MATCH_ID'])

[02/20/24 11:21:24] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=509758;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=825176;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=615324;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=460174;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=345703;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=979641;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=764288;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=212936;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=736056;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=256614;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:21:27] INFO     query execution done                                                     ]8;id=516157;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=234710;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS MATCHES_TEMP]                               ]8;id=495233;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=252428;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=952594;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=698439;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=623297;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=70807;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 248 rows processed.


### Time to deal with event data

In [123]:
game_ids

array([1729481, 1729482, 1729484, 1729486, 1729310, 1729313, 1729319,
       1729485, 1729316, 1729483], dtype=int64)

In [124]:
for i, val in enumerate(game_ids):
    if i == 0:
        ws_event_spadl = ws.read_events(output_fmt='spadl', match_id=[val])
    else:
        ws_event_spadl = pd.concat([ws_event_spadl, ws.read_events(output_fmt='spadl', match_id=[val])])

[02/20/24 11:21:42] INFO     Scraping game schedule from                                           ]8;id=500948;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=600480;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#427\427]8;;\
                             https://www.whoscored.com/Regions/252/Tournaments/2/Seasons/9618/Engl                 
                             and-Premier-League                                                                    

                    INFO     Scraping game schedule for Thursday, Feb 1 2024                       ]8;id=463508;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=18657;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Feb 3 2024                       ]8;id=383388;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=593621;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Sunday, Feb 4 2024                         ]8;id=846978;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=337844;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/20/24 11:21:43] INFO     Scraping game schedule for Monday, Feb 5 2024                         ]8;id=331761;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=496346;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

...

                    INFO     Scraping game schedule for Friday, Aug 25 2023                        ]8;id=228633;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=654257;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Saturday, Aug 26 2023                      ]8;id=81382;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=832559;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

                    INFO     Scraping game schedule for Sunday, Aug 27 2023                        ]8;id=597067;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=158022;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#327\327]8;;\

[02/20/24 11:31:24] INFO     [1/1] Retrieving game with id=1729483                                 ]8;id=563373;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py\whoscored.py]8;;\:]8;id=445137;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\whoscored.py#693\693]8;;\

d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\socceraction\spadl\opta.py:219: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  ).bfill()


In [181]:
# ws_event_spadl = ws.read_events(output_fmt='spadl')

In [125]:
ws_event_spadl.loc[ws_event_spadl['team'] == 'Manchester United', 'team'] = 'Manchester Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Newcastle United', 'team'] = 'Newcastle Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Newcastle', 'team'] = 'Newcastle Utd'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Tottenham Hotspur', 'team'] = 'Tottenham'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Wolverhampton Wanderers', 'team'] = 'Wolves'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Wolverhampton', 'team'] = 'Wolves'
ws_event_spadl.loc[ws_event_spadl['team'] == 'Nottingham Forest', 'team'] = "Nott'ham Forest"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Leicester', 'team'] = "Leicester City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Leeds', 'team'] = "Leeds United"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Sheffield United', 'team'] = "Sheffield Utd"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Sheff Utd', 'team'] = "Sheffield Utd"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Luton', 'team'] = "Luton Town"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Norwich', 'team'] = "Norwich City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Man City', 'team'] = "Manchester City"
ws_event_spadl.loc[ws_event_spadl['team'] == 'Man Utd', 'team'] = "Manchester Utd"

In [126]:
curr_team_names = fetch_data(cur, 'SELECT * FROM TEAMS')

[02/20/24 11:31:31] INFO     query: [SELECT * FROM TEAMS]                                             ]8;id=924978;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=70098;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=922932;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=488951;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 25                                     ]8;id=399452;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=656578;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

In [127]:
ws_event_spadl = ws_event_spadl.merge(spadl.actiontypes_df(), how="left").merge(spadl.bodyparts_df(), how="left")

In [128]:
ws_event_spadl.rename({'game_id':'WS_MATCH_ID', 'original_event_id':'ORIGINAL_EVENT_ID','period_id':'PERIOD_ID',
                       'time_seconds':'TIME_SECONDS', 'team_id':'TEAM_WS_ID', 'player_id':'PLAYER_WS_ID', 'start_x':'START_X',
                        'end_x':'END_X', 'start_y':'START_Y', 'end_y':'END_Y','type_name':'TYPE_NAME',
                        'bodypart_name':'BODYPART_NAME','result_id':'RESULT_ID', 'action_id':'ACTION_ID'}, axis=1, inplace=True)

In [129]:
ws_event_spadl = ws_event_spadl.merge(df_matches[['MATCH_ID', 'WS_MATCH_ID']], on='WS_MATCH_ID')

In [130]:
ws_event_spadl['WS_MATCH_ID'].value_counts()

WS_MATCH_ID
1729316    1879
1729319    1863
1729486    1812
1729485    1723
1729481    1641
1729482    1596
1729484    1522
1729483    1516
1729313    1508
1729310    1424
Name: count, dtype: int64

In [131]:
ws_event_spadl = ws_event_spadl.merge(curr_team_names[['TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_WS_ID')

In [132]:
ws_event_spadl.rename({'TEAM_FBREF_ID':'team_id','START_X':'start_x','START_Y':'start_y','END_X':'end_x','END_Y':'end_y'}, axis=1,
                      inplace=True)

In [133]:
df_matches_left = df_matches[df_matches['MATCH_ID'].isin(matches_to_scarpe)]

In [134]:
df_matches_left

,MATCH_ID,COMPETITION,SEASON,STADIUM,GAMEWEEK,DAY,HOME_TEAM_XG,AWAY_TEAM_XG,ATTENDANCE,REFEREE,WS_MATCH_ID,HOME_TEAM_ID,AWAY_TEAM_ID,DATE_TIME,HOME_TEAM_SCORE,AWAY_TEAM_SCORE
238,ca1e26a0,ENG-Premier League,2324,Gtech Community Stadium,25,Sat,1.6,2.8,17193,Michael Oliver,1729481,cd051869,822bd0ba,2024-02-17 12:30:00,1,4
239,59631171,ENG-Premier League,2324,Turf Moor,25,Sat,0.3,2.3,21311,Jarred Gillett,1729482,943e8050,18bb7c10,2024-02-17 15:00:00,0,5
240,796ff9f0,ENG-Premier League,2324,Craven Cottage,25,Sat,1.5,1.5,24401,Lewis Smith,1729484,fd962109,8602292d,2024-02-17 15:00:00,1,2
241,78fb7fc2,ENG-Premier League,2324,Etihad Stadium,25,Sat,2.6,1.4,52178,Andy Madley,1729486,b8fd03ef,cff3d9bb,2024-02-17 17:30:00,1,1
242,083c4444,ENG-Premier League,2324,St James' Park,25,Sat,2.2,2.2,52224,Michael Salisbury,1729310,b2b47a98,4ba7cbea,2024-02-17 15:00:00,2,2
243,d657db47,ENG-Premier League,2324,The City Ground,25,Sat,2.8,0.5,29490,Thomas Bramall,1729313,e4a775cb,7c21e445,2024-02-17 15:00:00,2,0
244,30193251,ENG-Premier League,2324,Tottenham Hotspur Stadium,25,Sat,1.6,1.8,61532,Anthony Taylor,1729319,361ca564,8cec06e1,2024-02-17 15:00:00,1,2
245,49523b04,ENG-Premier League,2324,Kenilworth Road Stadium,25,Sun,1.8,2.9,11483,David Coote,1729485,e297cd13,19538871,2024-02-18 16:30:00,1,2
246,811fa03f,ENG-Premier League,2324,Bramall Lane,25,Sun,0.6,2.9,29522,Stuart Attwell,1729316,1df6b87e,d07537b9,2024-02-18 14:00:00,0,5
247,573f2f77,ENG-Premier League,2324,Goodison Park,25,Mon,1.7,0.7,38164,Paul Tierney,1729483,d3fd31cc,47c64c55,2024-02-19 20:00:00,1,1


In [135]:
for index, row in df_matches_left.iterrows():
    inds_to_replace = ws_event_spadl[ws_event_spadl['MATCH_ID'] == row['MATCH_ID']].index
    ws_event_spadl.loc[inds_to_replace] = spadl.play_left_to_right(ws_event_spadl.loc[inds_to_replace], row['HOME_TEAM_ID'])

In [136]:
ws_event_spadl['NEW_TIME_SECONDS'] = ws_event_spadl['TIME_SECONDS']

In [137]:
ws_event_spadl_period1 = ws_event_spadl[ws_event_spadl['PERIOD_ID'] == 1]
ws_event_spadl_period2 = ws_event_spadl[ws_event_spadl['PERIOD_ID'] == 2]

In [138]:
start_time_2nd_half = ws_event_spadl_period2.groupby(['MATCH_ID', 'PERIOD_ID'])['NEW_TIME_SECONDS'].min().reset_index()

In [139]:
for index, row in tqdm(start_time_2nd_half.iterrows()):
    time_to_remove = row['NEW_TIME_SECONDS']
    ws_event_spadl.loc[(ws_event_spadl['MATCH_ID'] == row['MATCH_ID']) & (ws_event_spadl['PERIOD_ID'] == 2), 'NEW_TIME_SECONDS'] -= time_to_remove

10it [00:00, 401.03it/s]


In [140]:
ws_event_spadl.rename({'start_x':'START_X','end_x':'END_X','start_y':'START_Y','end_y':'END_Y',
                       'team_id':'TEAM_FBREF_ID'}, axis=1, inplace=True)

In [142]:
ws_event_spadl

,WS_MATCH_ID,ORIGINAL_EVENT_ID,PERIOD_ID,TIME_SECONDS,TEAM_WS_ID,PLAYER_WS_ID,START_X,END_X,START_Y,END_Y,type_id,RESULT_ID,bodypart_id,ACTION_ID,player,team,TYPE_NAME,BODYPART_NAME,MATCH_ID,TEAM_FBREF_ID,NEW_TIME_SECONDS
0,1729481,2647110317,1,0.0,26,345319.0,52.605,29.820,34.340,31.076,0,1,0,0,Alexis Mac Allister,Liverpool,pass,foot,ca1e26a0,822bd0ba,0.0
1,1729481,2647110337,1,3.0,26,345957.0,31.185,71.715,30.464,56.576,0,0,0,1,Ibrahima Konaté,Liverpool,pass,foot,ca1e26a0,822bd0ba,3.0
2,1729481,2647110367,1,5.0,189,372724.0,33.495,46.935,9.996,13.464,0,0,1,2,Nathan Collins,Brentford,pass,head,ca1e26a0,cd051869,5.0
3,1729481,2647110389,1,8.0,26,86829.0,63.105,74.970,51.408,51.952,0,0,1,3,Wataru Endo,Liverpool,pass,head,ca1e26a0,822bd0ba,8.0
4,1729481,2647110399,1,9.0,189,372724.0,27.720,30.345,13.804,5.984,0,1,1,4,Nathan Collins,Brentford,pass,head,ca1e26a0,cd051869,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16479,1729483,2648753777,2,3175.0,31,317896.0,6.090,6.090,5.236,5.236,9,1,0,1511,Jack Harrison,Everton,tackle,foot,573f2f77,d3fd31cc,3055.0
16480,1729483,NaN,2,3175.0,31,317896.0,6.090,5.145,5.236,2.380,21,1,0,1512,Jack Harrison,Everton,dribble,foot,573f2f77,d3fd31cc,3055.0
16481,1729483,2648753779,2,3175.0,31,317896.0,5.145,5.460,2.380,2.380,0,0,0,1513,Jack Harrison,Everton,pass,foot,573f2f77,d3fd31cc,3055.0
16482,1729483,2648753785,2,3179.0,162,67491.0,100.590,100.590,61.200,61.200,7,0,0,1514,Jordan Ayew,Crystal Palace,take_on,foot,573f2f77,47c64c55,3059.0


In [143]:
upsert_to_snowflake("EVENTS_SPADL", 
                    ws_event_spadl.drop(['type_id', 'player', 'team', 'WS_MATCH_ID', 'TEAM_WS_ID','bodypart_id'],axis=1), 
                    ['MATCH_ID','ACTION_ID'])

[02/20/24 11:32:46] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=646486;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=244493;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=830088;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=865351;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=917753;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=918031;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=688446;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=634901;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=267953;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=531975;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:32:51] INFO     query execution done                                                     ]8;id=50393;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=286897;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS EVENTS_SPADL_TEMP]                          ]8;id=872262;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=224582;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=358426;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=956166;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=622520;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=53502;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 16484 rows processed.


In [144]:
# cur.execute('SELECT * FROM EVENTS_SPADL')
# events_rows = cur.fetchall()
# column_names = [desc[0] for desc in cur.description]
# ws_event_spadl = pd.DataFrame(events_rows, columns=column_names)

### Update the "Players" table in SNowflake

In [145]:
## Sort out "Players" "Table
ws_players.rename({'team_id':'TEAM_WS_ID', 'game_id':'WS_MATCH_ID'}, axis=1, inplace=True)

In [146]:
ws_players = ws_players.merge(curr_team_names, on="TEAM_WS_ID", how="left")

In [147]:
ws_players = ws_players.sort_values(by=["WS_MATCH_ID","TEAM_NAME","jersey_number"])

In [148]:
fbref_lineups.merge(curr_team_names[['TEAM_NAME','TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_NAME').isna().sum()

jersey_number       0
player              0
TEAM_NAME           0
is_starter          0
position          104
minutes_played      0
TEAM_FBREF_ID       0
TEAM_WS_ID          0
dtype: int64

In [149]:
fbref_lineups = fbref_lineups.reset_index().merge(fbref_schedule.reset_index()[['game', 'game_id',
                                                                                'ws_game_id']], how='right', on='game')

In [150]:
fbref_lineups = fbref_lineups.rename({'ws_game_id':'WS_MATCH_ID'}, axis=1)

In [151]:
fbref_lineups = fbref_lineups.merge(curr_team_names[['TEAM_NAME','TEAM_FBREF_ID','TEAM_WS_ID']], on='TEAM_NAME')

In [152]:
ws_players.rename({'player_id':'WS_player_id', 'player_name':'WS_player_name','minutes_played':'WS_minutes_played',
                   'starting_position':'WS_starting_position', 'is_starter':'WS_is_starter'}, axis=1, inplace=True)

In [153]:
fbref_lineups.rename({'minutes_played':'FBREF_minutes_played', 'game_id':'MATCH_ID','position':'FBREF_position',
                      'is_starter':'FBREF_is_starter'}, axis=1, inplace=True)

In [154]:
fbref_lineups = fbref_lineups.merge(ws_players, on=["TEAM_FBREF_ID", "TEAM_NAME", "WS_MATCH_ID",
                                                    "TEAM_WS_ID", "jersey_number"])

In [155]:
df_players = fbref_lineups[['WS_player_id', 'player', 'WS_player_name']].drop_duplicates()

In [156]:
df_players.rename({'WS_player_id':'PLAYER_WS_ID', 'player':'PLAYER_FBREF_NAME', 'WS_player_name':'PLAYER_WS_NAME'}, 
                  axis=1, inplace=True)

In [157]:
df_players

,PLAYER_WS_ID,PLAYER_FBREF_NAME,PLAYER_WS_NAME
0,115131,Mark Flekken,Mark Flekken
1,101856,Christian Nørgaard,Christian Nørgaard
2,119822,Neal Maupay,Neal Maupay
3,339674,Mathias Jensen,Mathias Jensen
4,362275,Sergio Reguilón,Sergio Reguilón
...,...,...,...
395,409440,Naouirou Ahamada,Naouirou Ahamada
396,141246,Jaïro Riedewald,Jairo Riedewald
397,464285,David Ozoh,David Ozoh
398,517286,Franco Umeh-Chibueze,Franco Umeh-Chibueze


In [158]:
upsert_to_snowflake("PLAYERS", df_players, ['PLAYER_WS_ID'])

[02/20/24 11:33:27] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=121046;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=731941;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

[02/20/24 11:33:28] INFO     query execution done                                                     ]8;id=494345;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=390755;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=227827;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=368339;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=336590;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=754098;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=305858;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=813774;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:33:31] INFO     query execution done                                                     ]8;id=322790;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=272878;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS PLAYERS_TEMP]                               ]8;id=551747;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=826299;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=607413;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=424493;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=867922;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=665367;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 400 rows processed.


Player-match combos now

In [159]:
df_player_match = fbref_lineups[['WS_player_id', 'player','MATCH_ID', 'TEAM_FBREF_ID', 'WS_starting_position', 
                                 'FBREF_position','jersey_number','FBREF_minutes_played','FBREF_is_starter','WS_is_starter',
                                 'WS_minutes_played']]

In [160]:
df_player_match = df_player_match.rename({'WS_player_id':'PLAYER_WS_ID', 'player':'PLAYER_FBREF_NAME', 'WS_starting_position':'STARTING_POSITION_WS',
                        'FBREF_position':'POSITION_FBREF', 'jersey_number':'JERSEY_NUMBER', 'FBREF_minutes_played':'MINUTES_PLAYED_FBREF',
                        'FBREF_is_starter':'IS_STARTER_FBREF','WS_is_starter':'IS_STARTER_WS','WS_minutes_played':'MINUTES_PLAYED_WS'}, axis=1)

In [161]:
df_player_match

,PLAYER_WS_ID,PLAYER_FBREF_NAME,MATCH_ID,TEAM_FBREF_ID,STARTING_POSITION_WS,POSITION_FBREF,JERSEY_NUMBER,MINUTES_PLAYED_FBREF,IS_STARTER_FBREF,IS_STARTER_WS,MINUTES_PLAYED_WS
0,115131,Mark Flekken,ca1e26a0,cd051869,GK,GK,1,90,True,True,106
1,101856,Christian Nørgaard,ca1e26a0,cd051869,MC,CM,6,62,True,True,71
2,119822,Neal Maupay,ca1e26a0,cd051869,FW,FW,7,62,True,True,71
3,339674,Mathias Jensen,ca1e26a0,cd051869,MC,CM,8,73,True,True,82
4,362275,Sergio Reguilón,ca1e26a0,cd051869,DML,WB,12,84,True,True,93
...,...,...,...,...,...,...,...,...,...,...,...
395,409440,Naouirou Ahamada,573f2f77,47c64c55,Sub,AM,29,19,False,False,25
396,141246,Jaïro Riedewald,573f2f77,47c64c55,Sub,<NA>,44,0,False,False,0
397,464285,David Ozoh,573f2f77,47c64c55,Sub,CM,52,3,False,False,9
398,517286,Franco Umeh-Chibueze,573f2f77,47c64c55,Sub,<NA>,55,0,False,False,0


In [162]:
upsert_to_snowflake("PLAYER_MATCH", df_player_match, ['PLAYER_WS_ID', 'MATCH_ID'])

[02/20/24 11:33:39] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=408138;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=830096;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

                    INFO     query execution done                                                     ]8;id=348167;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=500606;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=886347;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=46045;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=266698;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=245696;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

[02/20/24 11:33:40] INFO     query execution done                                                     ]8;id=533010;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=382462;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:33:43] INFO     query execution done                                                     ]8;id=872689;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=469629;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS PLAYER_MATCH_TEMP]                          ]8;id=549203;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=624302;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=719139;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=379862;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=291895;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=21976;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 400 rows processed.


### Add Shot Event Data

In [163]:
matches_to_scarpe

['573f2f77',
 '083c4444',
 '59631171',
 '78fb7fc2',
 '796ff9f0',
 'ca1e26a0',
 'd657db47',
 '30193251',
 '49523b04',
 '811fa03f']

In [164]:
for i, val in enumerate(matches_to_scarpe):
    if i == 0:
        fbref_shot_events = fbref.read_shot_events(match_id=val)
    else:
        fbref_shot_events = pd.concat([fbref_shot_events, fbref.read_shot_events(match_id=val)])

[02/20/24 11:36:06] INFO     [1/1] Retrieving game with id=573f2f77                                   ]8;id=530210;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=587307;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:36:21] INFO     [1/1] Retrieving game with id=083c4444                                   ]8;id=888658;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=720309;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:36:37] INFO     [1/1] Retrieving game with id=59631171                                   ]8;id=471731;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=612016;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:36:53] INFO     [1/1] Retrieving game with id=78fb7fc2                                   ]8;id=634534;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=469246;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:37:08] INFO     [1/1] Retrieving game with id=796ff9f0                                   ]8;id=43669;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=389591;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:37:24] INFO     [1/1] Retrieving game with id=ca1e26a0                                   ]8;id=668941;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=83602;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:37:40] INFO     [1/1] Retrieving game with id=d657db47                                   ]8;id=300204;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=16797;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:37:56] INFO     [1/1] Retrieving game with id=30193251                                   ]8;id=514636;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=936510;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:38:11] INFO     [1/1] Retrieving game with id=49523b04                                   ]8;id=152032;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=176560;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

[02/20/24 11:38:27] INFO     [1/1] Retrieving game with id=811fa03f                                   ]8;id=626113;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py\fbref.py]8;;\:]8;id=589593;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\soccerdata\fbref.py#1057\1057]8;;\

In [165]:
fbref_shot_events = fbref_shot_events.rename_axis(index={'season': 'SEASON', 'league': 'COMPETITION'})

In [166]:
fbref_shot_events = make_season_integer(fbref_shot_events)

In [167]:
ws_event_spadl['TIME_SECONDS'] = ws_event_spadl['TIME_SECONDS'].astype(int)
ws_event_spadl['PLAYER_WS_ID'] = ws_event_spadl['PLAYER_WS_ID'].astype(int)

In [168]:
ws_shots = ws_event_spadl[(ws_event_spadl['TYPE_NAME'] == 'shot') | (ws_event_spadl['TYPE_NAME'] == 'shot_freekick') | \
               (ws_event_spadl['TYPE_NAME'] == 'shot_penalty')]

In [169]:
fbref_shot_events.reset_index(inplace=True)

In [170]:
fbref_shot_events.columns = fbref_shot_events.columns.get_level_values(0) + fbref_shot_events.columns.get_level_values(1)

In [171]:
fbref_shot_events = fbref_shot_events.merge(fbref_schedule[['COMPETITION','SEASON','game','game_id','ws_game_id']], 
                        on=['COMPETITION','SEASON','game'])

In [172]:
fbref_shot_events.rename({'player':'PLAYER_FBREF_NAME'}, axis=1, inplace=True)

In [173]:
fbref_shot_events['PLAYER_FBREF_NAME'] = fbref_shot_events['PLAYER_FBREF_NAME'].str.replace(' \(pen\)', '', regex=True)

In [174]:
fbref_shot_events = fbref_shot_events.merge(df_players, on='PLAYER_FBREF_NAME', how="left")

In [175]:
fbref_shot_events.rename({'game_id':'MATCH_ID'}, axis=1, inplace=True)

In [176]:
# ws_shots = ws_event_spadl[(ws_event_spadl['TYPE_NAME'] == 'shot') | (ws_event_spadl['TYPE_NAME'] == 'shot_freekick') | \
#                (ws_event_spadl['TYPE_NAME'] == 'shot_penalty')]

In [177]:
ws_dropped_shots = ws_shots[ws_shots['PLAYER_WS_ID'] == 444304]
ws_shots = ws_shots[ws_shots['PLAYER_WS_ID'] != 444304]

In [178]:
## Rogue Joel Veltman shot should be dropped
ws_shots = ws_shots.drop(ws_shots[(ws_shots['PLAYER_WS_ID'] == 110290) & (ws_shots['MATCH_ID'] == '36be5cee')].index)

In [179]:
for game in ws_shots['MATCH_ID'].unique():
    if len(ws_shots.loc[ws_shots.MATCH_ID == game]) != len(fbref_shot_events.loc[fbref_shot_events.MATCH_ID == game]):
        print(game)

In [180]:
# fbref_shot_events[['new_minute', 'added_minute']] = fbref_shot_events['minute'].apply(
#     lambda x: pd.Series(str(x).split('+') if '+' in str(x) else [x, 0])
# )

In [181]:
# fbref_shot_events['new_minute'] = fbref_shot_events['new_minute'].astype(int)
# fbref_shot_events['added_minute'] = fbref_shot_events['added_minute'].astype(int)

In [182]:
# fbref_shot_events = fbref_shot_events.sort_values(by=['MATCH_ID', 'new_minute', 'added_minute', 'PLAYER_WS_ID']).reset_index(drop=True)

In [183]:
ws_shots = ws_shots.sort_values(by=['MATCH_ID', 'PERIOD_ID','ACTION_ID']).reset_index(drop=True)

In [184]:
ws_shots['xG'] = np.nan
ws_shots['PSxG'] = np.nan
ws_shots['outcome'] = np.nan
ws_shots['distance'] = np.nan
ws_shots['body_part'] = np.nan
ws_shots['notes'] = np.nan
ws_shots['SCA 1player'] = np.nan
ws_shots['SCA 1event'] = np.nan
ws_shots['SCA 2player'] = np.nan
ws_shots['SCA 2event'] = np.nan
ws_shots['FBREF_PLAYER_WS_ID'] = np.nan
ws_shots['PLAYER_WS_NAME'] = np.nan

In [185]:
fbref_shot_events.rename({'PLAYER_WS_ID':'FBREF_PLAYER_WS_ID'}, axis=1, inplace=True)

In [186]:
cols_of_interest = ['xG', 'PSxG', 'outcome', 'distance', 'body_part', 'notes', 'SCA 1player', 'SCA 1event', 
                    'SCA 2player', 'SCA 2event', 'FBREF_PLAYER_WS_ID', 'PLAYER_WS_NAME']
for game in ws_shots['MATCH_ID'].unique():
    df_hold = fbref_shot_events[fbref_shot_events['MATCH_ID'] == game]
    ind_list = ws_shots[ws_shots['MATCH_ID'] == game].index
    ws_shots.loc[ind_list, cols_of_interest] = np.array(df_hold[cols_of_interest])

C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\3612877618.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[<NA>, <NA>, 0.16, 0.73, <NA>, <NA>, <NA>, 0.39, 0.03, <NA>, 0.05, <NA>, <NA>, <NA>, 0.93, 0.99, <NA>, <NA>, <NA>, <NA>, 0.08, <NA>, 0.33, <NA>, 0.99, <NA>, 0.09, <NA>, <NA>]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ws_shots.loc[ind_list, cols_of_interest] = np.array(df_hold[cols_of_interest])
C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\3612877618.py:6: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Off Target', 'Off Target', 'Saved', 'Saved', 'Off Target', 'Blocked', 'Off Target', 'Saved', 'Saved', 'Off Target', 'Saved', 'Off Target', 'Off Target', 'Off Target', 'Goal', 'Goal', 'Off Target', 'Off Target', 'Off Target', 'Off Target', 'Goal', 'Blocked', 'S

In [187]:
ws_shots['FBREF_PLAYER_WS_ID'] = ws_shots['FBREF_PLAYER_WS_ID'].astype(int)

In [191]:
mismatched_inds = ws_shots[ws_shots['PLAYER_WS_ID'] != ws_shots['FBREF_PLAYER_WS_ID']].index

In [192]:
len(mismatched_inds)

0

In [190]:
for i in range(len(mismatched_inds) // 2):
    val1 = ws_shots.loc[mismatched_inds, cols_of_interest].iloc[2*i]
    val2 = ws_shots.loc[mismatched_inds, cols_of_interest].iloc[(2*i)+1]

    ws_shots.loc[val1.name, cols_of_interest] = np.array(val2)
    ws_shots.loc[val2.name, cols_of_interest] = np.array(val1)

In [194]:
#### Remove this part later becasue it's just for Yarmolyuik

df_shots = ws_shots[['MATCH_ID','ACTION_ID','xG', 'PSxG', 'outcome', 'distance', 
          'body_part', 'notes', 'SCA 1player', 'SCA 1event', 'SCA 2player', 'SCA 2event']]

# ws_dropped_shots[[cols_of_interest]] = np.nan

# rows_to_add = [
#     [0.04,np.nan,'Blocked',16,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.03,0.12,'Saved',27,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.05,np.nan,'Off Target',21,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.49,0.25,'Saved',11,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.09,np.nan,'Off Target',8,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk'],
# [0.02,0.02,'Saved',19,'Right Foot',np.nan,np.nan,np.nan,np.nan,np.nan,444304,'Yehor Yarmoliuk']
# ]

# ws_dropped_shots[cols_of_interest] = np.array(rows_to_add)

In [195]:
# df_shots = pd.concat([df_shots, ws_dropped_shots[['MATCH_ID','ACTION_ID','xG', 'PSxG', 'outcome', 'distance', 
#           'body_part', 'notes', 'SCA 1player', 'SCA 1event', 'SCA 2player', 'SCA 2event']]]).sort_index()

In [196]:
df_shots.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
df_shots.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\3221164904.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shots.replace({"Jarrell Quansah": "Jarell Quansah"}, inplace=True)
C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\3221164904.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shots.replace({"Omotayo Adaramola": "Tayo Adaramola"}, inplace=True)


In [197]:
df_shots.rename({'xG':'XG', 'PSxG':'PSXG', 'outcome':'OUTCOME', 'distance':'DISTANCE', 'body_part':'BODY_PART',
                 'notes':'NOTES', 'SCA 1player':'SCA_1_PLAYER_FBREF_NAME', 'SCA 1event':'SCA_1_PLAYER_FBREF_EVENT',
                 'SCA 2player':'SCA_2_PLAYER_FBREF_NAME', 'SCA 2event':'SCA_2_PLAYER_FBREF_EVENT'}, axis=1, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\2795869585.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_shots.rename({'xG':'XG', 'PSxG':'PSXG', 'outcome':'OUTCOME', 'distance':'DISTANCE', 'body_part':'BODY_PART',


In [198]:
df_players.rename({'PLAYER_FBREF_NAME':'SCA_1_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
df_players.rename({'PLAYER_WS_ID':'SCA_1_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [199]:
df_shots = df_shots.merge(df_players[['SCA_1_PLAYER_FBREF_ID','SCA_1_PLAYER_FBREF_NAME']], on='SCA_1_PLAYER_FBREF_NAME', how='left')

In [200]:
df_players.rename({'SCA_1_PLAYER_FBREF_NAME':'SCA_2_PLAYER_FBREF_NAME'}, axis=1, inplace=True)
df_players.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_2_PLAYER_FBREF_ID'}, axis=1, inplace=True)

In [201]:
df_shots = df_shots.merge(df_players[['SCA_2_PLAYER_FBREF_ID','SCA_2_PLAYER_FBREF_NAME']], on='SCA_2_PLAYER_FBREF_NAME', how='left')

In [202]:
df_shots = df_shots.drop(['SCA_1_PLAYER_FBREF_NAME', 'SCA_2_PLAYER_FBREF_NAME'], axis=1)

In [203]:
df_shots.rename({'SCA_1_PLAYER_FBREF_ID':'SCA_1_PLAYER_WS_ID', 'SCA_2_PLAYER_FBREF_ID':'SCA_2_PLAYER_WS_ID'}, 
                axis=1, inplace=True)

In [204]:
df_shots.replace({pd.NA: np.nan}, inplace=True)

C:\Users\ksbha\AppData\Local\Temp\ipykernel_34656\3696910441.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_shots.replace({pd.NA: np.nan}, inplace=True)


In [205]:
df_shots['XG'] = df_shots['XG'].astype(float)
df_shots['PSXG'] = df_shots['PSXG'].astype(float)
df_shots['DISTANCE'] = df_shots['DISTANCE'].astype(int)

In [206]:
upsert_to_snowflake("SHOT_EVENTS", df_shots, ['MATCH_ID', 'ACTION_ID'])

[02/20/24 11:44:13] INFO     query: [CREATE TEMP STAGE /*                                             ]8;id=245193;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=45476;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             Python:snowflake.connector.pandas_tools.write_pandas() */ "...]                       

[02/20/24 11:44:14] INFO     query execution done                                                     ]8;id=910644;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=320783;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=901323;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=738067;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

                    INFO     query: [PUT /* Python:snowflake.connector.pandas_tools.write_pandas() */ ]8;id=576978;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=493159;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\
                             'file://C:\\Use...]                                                                   

                    INFO     query execution done                                                     ]8;id=414433;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=59292;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

...

[02/20/24 11:44:18] INFO     query execution done                                                     ]8;id=750284;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=774844;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     query: [DROP TABLE IF EXISTS SHOT_EVENTS_TEMP]                           ]8;id=89227;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=499419;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#833\833]8;;\

                    INFO     query execution done                                                     ]8;id=381052;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=894264;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#846\846]8;;\

                    INFO     Number of results in first chunk: 1                                      ]8;id=42521;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py\cursor.py]8;;\:]8;id=907961;file://d:\Users\ksbha\miniconda3\envs\footy\Lib\site-packages\snowflake\connector\cursor.py#988\988]8;;\

Upsert completed. 308 rows processed.


In [207]:
df_shots

,MATCH_ID,ACTION_ID,XG,PSXG,OUTCOME,DISTANCE,BODY_PART,NOTES,SCA_1_PLAYER_FBREF_EVENT,SCA_2_PLAYER_FBREF_EVENT,SCA_1_PLAYER_WS_ID,SCA_2_PLAYER_WS_ID
0,083c4444,39,0.01,NaN,Off Target,24,Right Foot,Volley,Pass (Live),Pass (Dead),128882.0,344314.0
1,083c4444,64,0.19,NaN,Off Target,4,Head,NaN,Pass (Live),Pass (Live),83078.0,342811.0
2,083c4444,217,0.02,0.16,Saved,33,Left Foot,NaN,Pass (Live),Pass (Live),128882.0,136456.0
3,083c4444,233,0.40,0.73,Saved,14,Left Foot,NaN,Pass (Live),Pass (Live),331579.0,136456.0
4,083c4444,259,0.04,NaN,Off Target,18,Right Foot,NaN,Pass (Live),Pass (Live),474264.0,349669.0
...,...,...,...,...,...,...,...,...,...,...,...,...
303,d657db47,1382,0.02,0.14,Saved,26,Left Foot,NaN,Pass (Live),Pass (Live),125004.0,105172.0
304,d657db47,1399,0.06,NaN,Blocked,16,Left Foot,Volley,Pass (Live),Pass (Live),399250.0,332867.0
305,d657db47,1461,0.35,NaN,Off Target,16,Right Foot,NaN,Pass (Live),Pass (Live),350088.0,399250.0
306,d657db47,1469,0.01,NaN,Blocked,31,Left Foot,NaN,Take-On,Pass (Live),400166.0,143595.0
